# 2. Graph Feature Engineering

# Import data

In [1]:
#!pip install torch
#!pip install torch_geometric

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from functools import reduce
from sklearn.preprocessing import LabelEncoder
import torch

In [3]:
# Import data tables
cash1 = pd.read_csv('cash1.csv').iloc[:, 1:]
emt2 = pd.read_csv('emt2.csv').iloc[:, 1:]
wire2 = pd.read_csv('wire2.csv').iloc[:, 1:]
kyc = pd.read_csv("kyc.csv")

# Data Segmentation

Since the data is very large, we will be performing unsupervised learning with GNN only on the high-risk labeled individuals (HRIs) and the customers they have had transactions with. To segement the data we will:
- Retrieve the customers who have been labeled 1 in the kyc dataframe
- Find all wire and emt transactions related to the HRI
- Keep all customer data of HRI and those related to HRI

In [4]:
# Get a list of HRI
HRIs = list(kyc[kyc['label'] == 1]['cust_id'].unique())

In [5]:
# Show the total number of HRIs
len(HRIs)

5514

In [6]:
# Create a set for the customers to include in our final data
focus_customers = set(HRIs)

In [7]:
# Create a function to the the transactions related to selected customers and add them to the focus_customers set
def expand_net(df, focus, focus_customers):
    """
    """
    # find transactions related to selected customers
    HR_trxns = df[(df['id sender'].isin(focus)) | (df['id receiver'].isin(focus))]
    
    # Get the customers related to the focus and add them to the focus_customers set
    sends = set(HR_trxns['id sender'].unique())
    rece = set(HR_trxns['id receiver'].unique())
    
    # return the updated focus customers
    return focus_customers.union(sends).union(rece)

In [8]:
# Add those related by wire 
focus_customers = expand_net(wire2, HRIs, focus_customers)
len(focus_customers)

19410

In [9]:
# Add those related by emt
focus_customers = expand_net(emt2, HRIs, focus_customers)
len(focus_customers)

67355

In [10]:
# Convert the set of focus customers to a list
fc = list(focus_customers)

In [11]:
len(fc)

67355

#### Show the focused datasets

The segmented dataframes we will be using for our GNNs will be:
- HR_wires: df with info about wires of HRI and those related to HRI
- HR_emts: df with info about emts of HRI and those related to HRI
- HR_cash: df with cash info for HRI and those related to HRI
- HRI_kyc: df with info on HRI and those related to HRI 

In [12]:
# Show the wire transactions for HRI and those related to HRI
HR_wires = wire2[(wire2['id sender'].isin(fc)) | (wire2['id receiver'].isin(fc))]
HR_wires

,id sender,id receiver,name sender,name receiver,wire value,country sender,country receiver,trxn_id,sender_Name,sender_Gender,sender_Occupation,sender_Age,sender_Tenure,sender_label,receiver_Name,receiver_Gender,receiver_Occupation,receiver_Age,receiver_Tenure,receiver_label
1,CUST43146787,CUST94438297,DR. BENJAMÍN PAREDES,BOBBY SERRANO,1267.0,CA,CA,LWCS42954834,DR. BENJAMÍN PAREDES,female,Architect,34.0,0.0,0.0,BOBBY SERRANO,male,Corporate Executive,45.0,21.0,0.0
2,CUST82396415,EXTERNAL842611,WHITNEY WRIGHT,VICTORIA HOGAN,8591.0,CA,US,NTTG55749308,WHITNEY WRIGHT,female,"Freelancer (e.g., Graphic Designer, Writer)",38.0,5.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
3,EXTERNAL851271,CUST84545757,DR.TRACY MOODY,MINDY BURGESS,1480.5,CA,CA,IXVD84599097,NaN,NaN,NaN,NaN,NaN,NaN,MINDY BURGESS,female,Construction Contractor,56.0,21.0,0.0
5,CUST61153056,EXTERNAL567325,DUANE HENDERSON,MICHAEL OSBORNE,1587.0,CA,CA,SLBV29462341,DUANE HENDERSON,male,Student,22.0,3.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
6,CUST93918261,CUST16525706,VINCENT BÉDARD,ALLA LEONOVNA MUHINA,1546.0,CA,CA,ERLU26785367,VINCENT BÉDARD,male,Baker,36.0,0.0,1.0,ALLA LEONOVNA MUHINA,female,Other,61.0,13.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67866,EXTERNAL304991,CUST86403791,LIAO YING,DR.HUGUES NADEAU,6059.5,CA,CA,LRRP66624765,NaN,NaN,NaN,NaN,NaN,NaN,DR.HUGUES NADEAU,male,Wedding Planner,35.0,12.0,0.0
67867,CUST89899937,EXTERNAL470484,ÉLISABETH BENOÎT,DANIEL SHEPPARD,5067.0,CA,CA,KVQK50168638,ÉLISABETH BENOÎT,female,Student,25.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
67868,CUST59834819,EXTERNAL634239,EMMA RICHARD,WANG HUA,18874.0,CA,CN,IUIP17370739,EMMA RICHARD,female,Stock Broker,44.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
67870,CUST31344256,CUST78691452,DR. JANET DIAZ,TIMOTHY BENNETT,4084.0,CA,CA,ZHVK78574815,DR. JANET DIAZ,female,Tour Guide,34.0,5.0,1.0,TIMOTHY BENNETT,other,Radiologist,42.0,0.0,0.0


In [13]:
# Show the emt transactions for HRI and those related to HRI
HR_emts = emt2[(emt2['id sender'].isin(fc)) | (emt2['id receiver'].isin(fc))]
HR_emts

,id sender,id receiver,name sender,name receiver,emt message,emt value,trxn_id,sender_Name,sender_Gender,sender_Occupation,sender_Age,sender_Tenure,sender_label,receiver_Name,receiver_Gender,receiver_Occupation,receiver_Age,receiver_Tenure,receiver_label
1,EXTERNAL623153,CUST59533929,GINA WISE,BRIAN HAMILTON,NaN,1170.5,RAUG63886259,NaN,NaN,NaN,NaN,NaN,NaN,BRIAN HAMILTON,male,Import/Export Dealer,37.0,6.0,0.0
3,CUST59096559,EXTERNAL470507,KEVIN PARK,FREDERICK CARPENTER,NaN,46.0,WPXP45854083,KEVIN PARK,male,Import/Export Business Owner,34.0,8.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
5,CUST27403977,CUST14798197,IND.DAVID DUNLAP JR.,FRÉDÉRIC-BERTRAND DROUIN,NaN,480.0,TRNT55099512,IND.DAVID DUNLAP JR.,other,Private Security Company Owner,69.0,14.0,0.0,FRÉDÉRIC-BERTRAND DROUIN,male,Construction Worker,22.0,4.0,0.0
7,CUST65275585,CUST26294363,EDUARDO PONCE VILLAREAL,CHERYL BENNETT,NaN,735.0,YSNV62579819,EDUARDO PONCE VILLAREAL,female,Architect,44.0,14.0,1.0,CHERYL BENNETT,female,Distiller,49.0,10.0,0.0
8,EXTERNAL483303,CUST65962607,JAMES RUBIO,JACQUELINE THIBODEAU-ST-PIERRE,NaN,540.0,MZYI28216959,NaN,NaN,NaN,NaN,NaN,NaN,JACQUELINE THIBODEAU-ST-PIERRE,female,Lawyer,48.0,3.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
506444,CUST54995802,CUST75573960,STACEY CLARKE,BRYAN JOHNSON,NaN,682.0,XAOG83079223,STACEY CLARKE,female,Economist,34.0,3.0,0.0,BRYAN JOHNSON,male,Dentist,42.0,16.0,0.0
506446,EXTERNAL484002,CUST74979363,ALEXANDRA GRANT,WANDA HAYDEN,NaN,119.0,USHN74907347,NaN,NaN,NaN,NaN,NaN,NaN,WANDA HAYDEN,female,Cashier,27.0,1.0,0.0
506447,CUST12188131,CUST68693554,JASMIN SMITH,MICHAEL CRUZ,NaN,208.0,VXES44436032,JASMIN SMITH,female,Event Planner,46.0,9.0,0.0,MICHAEL CRUZ,male,Unknown,35.0,0.0,0.0
506448,EXTERNAL470333,CUST90504001,NOAH BELL,MR. ERIC WALTERS,Fox racing motocross gear,150.0,LTUK21435620,NaN,NaN,NaN,NaN,NaN,NaN,MR. ERIC WALTERS,male,Real Estate Broker,29.0,5.0,0.0


In [14]:
# Create a dataframe for fc so that we can create a set without external users
fc_df = pd.DataFrame({'cust_id': fc})
fc_df.loc[(fc_df['cust_id'].str[:1] == 'E'), 'ext'] = 1
fc_df['ext'] = fc_df['ext'].fillna(0).astype(int)
only_custs = fc_df[fc_df['ext'] == 0]
oc = list(only_custs['cust_id'])
len(oc)

38571

In [15]:
# Show the cash transactions for HRI and those customers (no external users) related to HRI
HR_cash = cash1[cash1['cust_id'].isin(oc)]
HR_cash

,cust_id,amount,type,trxn_id,Name,Gender,Occupation,Age,Tenure,label,...,music,art,books,building,services,student,code,news,stats,vague
1,CUST78509707,4800,deposit,BFMG48785876,LUCY-OCÉANE VINCENT,female,Real Estate Agent,33.0,0.0,0,...,0,0,0,0,0,0,0,0,0,0
5,CUST83072236,7420,withdrawal,FUKV94845036,BILLY ANDRADE,male,Private Banker,48.0,17.0,1,...,0,0,0,0,0,0,0,0,0,0
11,CUST42415032,5595,deposit,NUZO58830551,LORI WHEELER,female,Private Banker,39.0,16.0,1,...,0,0,0,0,0,0,0,0,0,0
12,CUST38117029,1600,withdrawal,ZOSP34629709,MICHELLE BOULANGER,female,Architect,43.0,7.0,0,...,0,0,0,1,0,0,0,0,0,0
13,CUST67313614,1055,withdrawal,HQUJ43887606,MRS. LISA DAVIS MD,female,Art Historian,30.0,4.0,0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212518,CUST71021014,1140,deposit,JOQU43611104,DR.NANCY SMITH,female,Antique Dealer,48.0,2.0,0,...,0,0,0,0,0,0,0,0,0,0
212521,CUST85251436,6870,deposit,LTBH81014009,NATHAN JENSEN,male,Private Jet Broker,34.0,1.0,1,...,0,0,0,0,0,0,0,0,0,0
212525,CUST40749034,8740,withdrawal,GGHM25093698,DR.ROSS MAYER,male,Construction Contractor,22.0,4.0,0,...,0,0,0,1,0,0,0,0,0,0
212526,CUST26682704,5750,withdrawal,CNXP31340871,DR.BRITTANY LOPEZ,female,"Freelancer (e.g., Graphic Designer, Writer)",31.0,3.0,1,...,0,0,1,0,0,0,0,0,0,0


In [16]:
# Get the customer data for HRI and those related to HRI
HRI_kyc = kyc[kyc['cust_id'].isin(fc)]
HRI_kyc

,Name,Gender,Occupation,Age,Tenure,cust_id,label
0,JENNIFER WELLS,female,Architect,45.0,13.0,CUST82758793,0
2,DENISE LEWIS,female,Jewelry Dealer,43.0,11.0,CUST67222818,0
4,KYLE EDWARDS,male,Real Estate Broker,39.0,21.0,CUST76401392,1
8,BRIAN OLIVER,male,Antiques Dealer,47.0,10.0,CUST93206512,0
10,MAXIME VINCENT,male,Bus Driver,37.0,15.0,CUST23641762,0
...,...,...,...,...,...,...,...
195746,JOSHUA GIBSON,male,Carpenter,49.0,6.0,CUST82187581,0
195768,BILLY ELLIOTT,male,Filmmaker,35.0,0.0,CUST83165261,0
195783,WANG MIN,female,Private Equity Fund Manager,37.0,12.0,CUST67655265,1
195784,THOMAS YOUNG,male,Software Developer,46.0,3.0,CUST23014082,0


# Mapping Global customer ids

Since there are some customer ids that are are labeled as external in the transaction data, we will encode new labels for the id so that customers and external users can be identified across tables.

In [17]:
# Gather the customer ids from all tables: HRI_kyc, HR_wires1, HR_emts1, HR_cash
kyc_custs = set(HRI_kyc['cust_id'])
wire_sends = set(HR_wires['id sender'])
wire_recs = set(HR_wires['id receiver'])
emt_sends = set(HR_emts['id sender'])
emt_recs = set(HR_emts['id receiver'])
cash_custs = set(HR_cash['cust_id'])

In [18]:
# Get the union of all sets to get all unique customer ids across the tables 
all_ids = list(kyc_custs.union(wire_sends).union(wire_recs).union(emt_sends).union(emt_recs).union(cash_custs))
len(all_ids)

196058

In [19]:
# create a dataframe
custs_df = pd.DataFrame({'cust_id': all_ids})
custs_df

,cust_id
0,EXTERNAL697190
1,CUST78776608
2,EXTERNAL591520
3,EXTERNAL611545
4,EXTERNAL367321
...,...
196053,CUST59598987
196054,EXTERNAL769817
196055,EXTERNAL167932
196056,CUST53356223


In [20]:
# create a label encoder to encode the customer ids
cust_encoder = LabelEncoder()

# fit on the cust_id column
cust_encoder.fit(custs_df['cust_id'])

# create a new column of the newly encoded cust ids
custs_df['Global_id'] = cust_encoder.transform(custs_df['cust_id'])

In [21]:
# Create a flag for whether customer is an external user or not
custs_df.loc[(custs_df['cust_id'].str[:1] == 'E'), 'ext'] = 1
custs_df['ext'] = custs_df['ext'].fillna(0).astype(int)
custs_df

,cust_id,Global_id,ext
0,EXTERNAL697190,172634,1
1,CUST78776608,96489,0
2,EXTERNAL591520,164476,1
3,EXTERNAL611545,166008,1
4,EXTERNAL367321,146916,1
...,...,...,...
196053,CUST59598987,69545,0
196054,EXTERNAL769817,178181,1
196055,EXTERNAL167932,131539,1
196056,CUST53356223,60615,0


In [22]:
# update the HR_kyc with non-external customers in custs_df
non_ext = list(custs_df[custs_df['ext'] == 0]['cust_id'])
HRI_kyc = kyc[kyc['cust_id'].isin(non_ext)]
HRI_kyc

,Name,Gender,Occupation,Age,Tenure,cust_id,label
0,JENNIFER WELLS,female,Architect,45.0,13.0,CUST82758793,0
1,ANTHONY ADAMS,male,Musician,52.0,8.0,CUST69248708,0
2,DENISE LEWIS,female,Jewelry Dealer,43.0,11.0,CUST67222818,0
4,KYLE EDWARDS,male,Real Estate Broker,39.0,21.0,CUST76401392,1
5,BECKY SUTTON,female,Sommelier,26.0,8.0,CUST41866546,0
...,...,...,...,...,...,...,...
195783,WANG MIN,female,Private Equity Fund Manager,37.0,12.0,CUST67655265,1
195784,THOMAS YOUNG,male,Software Developer,46.0,3.0,CUST23014082,0
195785,CASEY JONES,male,Miner,31.0,10.0,CUST17691251,0
195786,NICOLE-CÉCILE LEBLANC,female,Unknown,21.0,3.0,CUST26444112,0


In [23]:
# create a seperate df for just the ids and gids
ids_df = custs_df.iloc[:, :-1]

Add `Global_id` to the existing dataframes

In [24]:
# Add the new id to kyc dataframe
HR_kyc = HRI_kyc.copy()
HR_kyc = pd.merge(HR_kyc, ids_df, how='left', on='cust_id')
# HR_kyc.head()

In [25]:
# Add the new id to the wire2 dataframe
HR_wires1 = HR_wires.copy().rename(columns={'id sender': 'cust_id'})
HR_wires1 = pd.merge(HR_wires1, ids_df, how='left', on='cust_id')
HR_wires1 = HR_wires1.rename(columns={'cust_id': 'id sender', 
                              'id receiver': 'cust_id', 
                              'Global_id':'sender_global_id'})
HR_wires1 = pd.merge(HR_wires1, ids_df, how='left', on='cust_id')
HR_wires1 = HR_wires1.rename(columns={'cust_id': 'id receiver', 
                              'Global_id':'rec_global_id'})
# HR_wires1.head()

In [26]:
# Add the new id to the emt dataframe
HR_emts1 = HR_emts.copy().rename(columns={'id sender': 'cust_id'})
HR_emts1 = pd.merge(HR_emts1, ids_df, how='left', on='cust_id')
HR_emts1 = HR_emts1.rename(columns={'cust_id': 'id sender', 
                            'id receiver': 'cust_id', 
                            'Global_id':'sender_global_id'})
HR_emts1 = pd.merge(HR_emts1, ids_df, how='left', on='cust_id')
HR_emts1 = HR_emts1.rename(columns={'cust_id': 'id receiver', 
                            'Global_id':'rec_global_id'})

# HR_emts1.head()

In [27]:
# Add the new id to the cash table
HR_cash1 = HR_cash.copy()
HR_cash1 = pd.merge(HR_cash1, ids_df, how='left', on='cust_id')
# HR_cash1.head()

# Feature Engineering

The following summaries the tables and new variables that will be created:

### Final DataFrames and variable names

`General Cust Info` DF: cust_info </br>
`Detailed Customer Info` DF: detailed_cust_info</br>
`Dci`: detailed_cust_info + Original Occupations (Used for Community Analysis) </br>
`External Users` DF: external_info</br>

The following lists outline the **new variables** we will be adding to each dataframe:

Abbreviations:
- **rec**: received
- **ext**: related to external users
- **int**: internally (customer to customer)
- **CA**: Canada
- **dom**: domestically
- **ctry**: country abbrev., e.g. CA, US

**Wire and EMT DataFrame Additions**
- **External/Internal Users Related**
    - `sender_ext`: Binary flag for if the sender of a transaction is an external user
    - `receiver_ext`:  Binary flag for if the receiver of a transaction is an external user
    
**Wire DataFrame Additions**
- **Location Related**
    - `in_CA`: Binary flag for if receiver is in CA but sender is not in CA
    - `out_CA`: Binary flag for if sender is in CA but receiver is not in CA

**Customer DataFrame Additions**
- `Country:` country of the customer 
- **Wire related**
    - `Num_wires_sent`, `Avg_wire_amt_sent`, 
    - `Num_wires_received`, `Avg_wire_amt_received`
    - `Num_wire_rec_ext`, `Avg_wire_rec_ext`
    - `Num_wire_sent_ext`, `Avg_wire_sent_ext`
    - `Num_wire_rec_int`, `Avg_wire_rec_int`
    - `Num_wire_sent_int`, `Avg_wire_sent_int`
    - `Num_wire_out_CA`, `Avg_wire_out_CA`
    - `Num_wire_in_CA`, `Avg_wire_in_CA`
    - `Num_wire_rec_dom`, `Avg_wire_rec_dom`
    - `Num_wire_sent_dom`, `Avg_wire_sent_dom`
    - `Num_emt_rec_ext`, `Avg_emt_rec_ext`
    - `Num_emt_sent_ext`, `Avg_emt_sent_ext`
    - `Num_emt_rec_int`, `Avg_emt_rec_int`
    - `Num_emt_sent_int`, `Avg_emt_sent_int`
- **Emt related**
    - `Num_emts_sent`, `Avg_emt_amt_sent`
    - `Num_emt_received`, `Avg_emt_amt_received`
    - `Num_emt_rec_ext`, `Avg_emt_rec_ext`
    - `Num_emt_sent_ext`, `Avg_emt_sent_ext`
    - `Num_emt_rec_int`,`Avg_emt_rec_int`
    - `Num_emt_sent_in`, `Avg_emt_sent_int`
- **Cash Related**
    - `Num_deposit`, `Avg_deposit`
    - `Num_withdrawal`, `Avg_withdrawal`

**External Users DataFrame**
- **Wire related**
    - `Num_wires_sent`, `Avg_wire_amt_sent`
    - `Num_wires_received`, `Avg_wire_amt_received`   
- **Emt related**
    - `Num_emts_sent`, `Avg_emt_amt_sent`       
    - `Num_emts_received`, `Avg_emt_amt_received`
- **Country Flags**
    - `Country_{Ctry}`: Binary variable, 1 if customer is from Country

### Identify External Users in HR_wire1 and HR_emt1

In [28]:
# create a function to identify whether an individual is an external user
def get_ext(df):
    """
    """
    # Create flags for whether the sender/receiver is an external individual
    df.loc[df['id sender'].str[:1] == 'E', 'sender_ext'] = 1
    df.loc[df['id receiver'].str[:1] == 'E', 'receiver_ext'] = 1
    df[['sender_ext', 'receiver_ext']] = df[['sender_ext', 'receiver_ext']].fillna(0).astype(int)
    df

Add `sender_ext`, `receiver_ext`, `no_ext` variables to HR_wires1 and HR_emts1

In [29]:
# Create flags for whether the sender/receiver is an external individual
get_ext(HR_wires1)
get_ext(HR_emts1)

In [30]:
# show the resulting wires df
HR_wires1.head()

,id sender,id receiver,name sender,name receiver,wire value,country sender,country receiver,trxn_id,sender_Name,sender_Gender,...,receiver_Name,receiver_Gender,receiver_Occupation,receiver_Age,receiver_Tenure,receiver_label,sender_global_id,rec_global_id,sender_ext,receiver_ext
0,CUST43146787,CUST94438297,DR. BENJAMÍN PAREDES,BOBBY SERRANO,1267.0,CA,CA,LWCS42954834,DR. BENJAMÍN PAREDES,female,...,BOBBY SERRANO,male,Corporate Executive,45.0,21.0,0.0,46393,118403,0,0
1,CUST82396415,EXTERNAL842611,WHITNEY WRIGHT,VICTORIA HOGAN,8591.0,CA,US,NTTG55749308,WHITNEY WRIGHT,female,...,NaN,NaN,NaN,NaN,NaN,NaN,101584,183844,0,1
2,EXTERNAL851271,CUST84545757,DR.TRACY MOODY,MINDY BURGESS,1480.5,CA,CA,IXVD84599097,NaN,NaN,...,MINDY BURGESS,female,Construction Contractor,56.0,21.0,0.0,184551,104548,1,0
3,CUST61153056,EXTERNAL567325,DUANE HENDERSON,MICHAEL OSBORNE,1587.0,CA,CA,SLBV29462341,DUANE HENDERSON,male,...,NaN,NaN,NaN,NaN,NaN,NaN,71803,162608,0,1
4,CUST93918261,CUST16525706,VINCENT BÉDARD,ALLA LEONOVNA MUHINA,1546.0,CA,CA,ERLU26785367,VINCENT BÉDARD,male,...,ALLA LEONOVNA MUHINA,female,Other,61.0,13.0,0.0,117672,9197,0,0


In [31]:
# show the resulting emts df
HR_emts1.head()

,id sender,id receiver,name sender,name receiver,emt message,emt value,trxn_id,sender_Name,sender_Gender,sender_Occupation,...,receiver_Name,receiver_Gender,receiver_Occupation,receiver_Age,receiver_Tenure,receiver_label,sender_global_id,rec_global_id,sender_ext,receiver_ext
0,EXTERNAL623153,CUST59533929,GINA WISE,BRIAN HAMILTON,NaN,1170.5,RAUG63886259,NaN,NaN,NaN,...,BRIAN HAMILTON,male,Import/Export Dealer,37.0,6.0,0.0,166925,69450,1,0
1,CUST59096559,EXTERNAL470507,KEVIN PARK,FREDERICK CARPENTER,NaN,46.0,WPXP45854083,KEVIN PARK,male,Import/Export Business Owner,...,NaN,NaN,NaN,NaN,NaN,NaN,68825,155054,0,1
2,CUST27403977,CUST14798197,IND.DAVID DUNLAP JR.,FRÉDÉRIC-BERTRAND DROUIN,NaN,480.0,TRNT55099512,IND.DAVID DUNLAP JR.,other,Private Security Company Owner,...,FRÉDÉRIC-BERTRAND DROUIN,male,Construction Worker,22.0,4.0,0.0,24521,6745,0,0
3,CUST65275585,CUST26294363,EDUARDO PONCE VILLAREAL,CHERYL BENNETT,NaN,735.0,YSNV62579819,EDUARDO PONCE VILLAREAL,female,Architect,...,CHERYL BENNETT,female,Distiller,49.0,10.0,0.0,77547,23015,0,0
4,EXTERNAL483303,CUST65962607,JAMES RUBIO,JACQUELINE THIBODEAU-ST-PIERRE,NaN,540.0,MZYI28216959,NaN,NaN,NaN,...,JACQUELINE THIBODEAU-ST-PIERRE,female,Lawyer,48.0,3.0,0.0,156121,78536,1,0


### Add Location Related Variables

Add `out_CA` and `in_CA` variables to HR_wires1

In [32]:
# Create another variable to indicate whether the transaction is sending money out of canada or sending money into canada
HR_wires1.loc[(HR_wires1['country sender'] == 'CA') & (HR_wires1['country receiver'] != 'CA'), 'out_CA'] = 1
HR_wires1.loc[(HR_wires1['country receiver'] == 'CA') & (HR_wires1['country sender'] != 'CA'), 'in_CA'] = 1
HR_wires1[['out_CA', 'in_CA']] = HR_wires1[['out_CA', 'in_CA']].fillna(0).astype(int)

## Extracting Customer data

### Extract customer's country

In [33]:
# Groupby sender and get country info
send_ctrs = pd.DataFrame(HR_wires1[['sender_global_id', 'country sender']].groupby(['sender_global_id']).sum()).reset_index()
send_ctrs['country'] = send_ctrs['country sender'].str[-2:]

# save the id and country as a tuple, then create a set
sids = list(send_ctrs['sender_global_id'])
s_ctrs = list(send_ctrs['country'])

sc_info = { (sids[x], s_ctrs[x]) for x in range(len(send_ctrs)) }
len(sc_info)

30922

In [34]:
# Groupby receiver and get country info 
rec_ctrs = pd.DataFrame(HR_wires1[['rec_global_id', 'country receiver']].groupby(['rec_global_id']).sum()).reset_index()
rec_ctrs['country'] = rec_ctrs['country receiver'].str[-2:]

# save the id and country as a tuple, then create a set
rids = list(rec_ctrs['rec_global_id'])
r_ctrs = list(rec_ctrs['country'])

rc_info = { (rids[x], r_ctrs[x]) for x in range(len(rec_ctrs)) }
len(rc_info)

30139

In [35]:
# Merge the two sets to get all customer info
all_ctry_info = list(sc_info.union(rc_info))
len(all_ctry_info)

46436

In [36]:
# Extract the data from the set and convert into a dataframe
gids = [ x[0] for x in all_ctry_info ]
ctys = [ x[1] for x in all_ctry_info ]

cust_ctrys = pd.DataFrame({'Global_id': gids, 'Country': ctys})
cust_ctrys

,Global_id,Country
0,41847,CA
1,153141,CA
2,9951,CA
3,64937,CA
4,120453,CA
...,...,...
46431,108302,CA
46432,187170,CA
46433,76668,CA
46434,19153,CA


For the customers who had no wire transactions, append 'Missing'

In [37]:
# Get all the global ids in the custs_df
all_gids = set(custs_df['Global_id'])
len(all_gids)

196058

In [38]:
# Get the customers who had no wire transactions
checked = set(cust_ctrys['Global_id'])
no_wires = list(all_gids.difference(checked))
len(no_wires)

149622

In [39]:
# Create a dataframe for those missing the country and concat to cust_ctrys
miss_c = ['Missing'] * len(no_wires)
missing_ctry = pd.DataFrame({'Global_id':no_wires, 'Country': miss_c})
cust_ctrys = pd.concat([cust_ctrys, missing_ctry])

# Merge with custs_df to get ext flag 
cust_ctrys = pd.merge(cust_ctrys, custs_df, on='Global_id').drop(columns='cust_id')
cust_ctrys

,Global_id,Country,ext
0,41847,CA,0
1,153141,CA,1
2,9951,CA,0
3,64937,CA,0
4,120453,CA,0
...,...,...,...
196053,196051,Missing,1
196054,196052,Missing,1
196055,196054,Missing,1
196056,196056,Missing,1


### Add country dummies to HR_wires1

In [40]:
# get dummies for the country sender and country receiver columns 
HR_wires1 = pd.get_dummies(HR_wires1, columns=['country sender', 'country receiver'], dtype=int)

In [41]:
HR_wires1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48292 entries, 0 to 48291
Data columns (total 44 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id sender            48292 non-null  object 
 1   id receiver          48292 non-null  object 
 2   name sender          48292 non-null  object 
 3   name receiver        48292 non-null  object 
 4   wire value           48292 non-null  float64
 5   trxn_id              48292 non-null  object 
 6   sender_Name          27448 non-null  object 
 7   sender_Gender        27448 non-null  object 
 8   sender_Occupation    27448 non-null  object 
 9   sender_Age           27448 non-null  float64
 10  sender_Tenure        27448 non-null  float64
 11  sender_label         27448 non-null  float64
 12  receiver_Name        35512 non-null  object 
 13  receiver_Gender      35512 non-null  object 
 14  receiver_Occupation  35512 non-null  object 
 15  receiver_Age         35512 non-null 

### Construct Dataframe for General Customer Wire Transactions

In [42]:
# Get a list of all customers that are not external users from custs_df
no_xcusts = set(custs_df[custs_df['ext'] == 0]['Global_id'])
len(no_xcusts)

126183

In [43]:
# create a function to find the customers that did not have any amount for the given transaction
def find_custs(df):
    """
    """
    # get the customers that were recorded
    recorded = set(df['Global_id'])
    
    # find the set difference between the non-exteral customer gids and the recorded ones
    not_recorded = list(no_xcusts.difference(recorded))
    
    # create 0s to fill in the num and sum columns
    fills = [0] * len(not_recorded)
    
    # create a dataframe and append to existing dataframe
    nr_df = pd.DataFrame({'Global_id': not_recorded, 'Num': fills, 'Avg': fills})
    
    return pd.concat([df, nr_df])

#### Total Count and Average Wire Amt Sent by Customers

In [44]:
# filter by non-external sender to create a groupby to 
# get the total amount of wires sent by each customer
sent = HR_wires1[HR_wires1['sender_ext'] == 0]
sent_w = pd.DataFrame(sent.groupby(['sender_global_id']).agg(Num=('id receiver', np.count_nonzero), 
                                                             Avg=('wire value', np.mean))).reset_index().rename(columns={'sender_global_id': 'Global_id'})
sent_w = find_custs(sent_w)
sent_w

,Global_id,Num,Avg
0,4,4,2332.625
1,7,1,5046.000
2,15,1,1420.500
3,18,1,7251.500
4,21,1,24687.500
...,...,...,...
109221,126177,0,0.000
109222,126178,0,0.000
109223,126179,0,0.000
109224,126180,0,0.000


#### Total Count and Average Wire Amount Received by Customer

In [45]:
#  filter by non-external receiver to
# create a groupby to get the total amount of wires recived by each customer
rec = HR_wires1[HR_wires1['receiver_ext'] == 0]
rec_w = pd.DataFrame(rec.groupby(['rec_global_id']).agg(Num=('id sender', np.count_nonzero), 
                                                             Avg=('wire value', np.mean))).reset_index().rename(columns={'rec_global_id': 'Global_id'})
rec_w = find_custs(rec_w)
rec_w

,Global_id,Num,Avg
0,1,1,1526.000000
1,4,3,3623.000000
2,6,3,6547.000000
3,7,2,5787.500000
4,18,3,5642.833333
...,...,...,...
105978,126177,0,0.000000
105979,126178,0,0.000000
105980,126179,0,0.000000
105981,126180,0,0.000000


In [46]:
# Merge the dataframes together
gen_wireinfo = pd.merge(sent_w, rec_w, how='left', on='Global_id')
cols = ['Global_id', 'Num_wires_sent', 'Avg_wire_amt_sent', 'Num_wires_received', 'Avg_wire_amt_received']
gen_wireinfo.columns = cols
gen_wireinfo

,Global_id,Num_wires_sent,Avg_wire_amt_sent,Num_wires_received,Avg_wire_amt_received
0,4,4,2332.625,3,3623.000000
1,7,1,5046.000,2,5787.500000
2,15,1,1420.500,0,0.000000
3,18,1,7251.500,3,5642.833333
4,21,1,24687.500,0,0.000000
...,...,...,...,...,...
126178,126177,0,0.000,0,0.000000
126179,126178,0,0.000,0,0.000000
126180,126179,0,0.000,0,0.000000
126181,126180,0,0.000,0,0.000000


### Construct Dataframe for Specific Customer Wire Transactions

#### Total Count and Average Wire Amount Received from External Users

In [47]:
# Create a groupby customerid to get the amount received from external users for each customer
ext_sendw = HR_wires1[HR_wires1['sender_ext'] == 1]
received_ext = pd.DataFrame(ext_sendw.groupby(['rec_global_id']).agg(Num=('id sender', np.count_nonzero), 
                                                              Avg=('wire value', np.mean))).reset_index().rename(columns={'rec_global_id': 'Global_id'})

received_ext = find_custs(received_ext)
received_ext

,Global_id,Num,Avg
0,4,2,4814.50
1,6,2,6207.50
2,7,1,7795.00
3,18,2,5239.25
4,47,1,1974.00
...,...,...,...
111371,126177,0,0.00
111372,126178,0,0.00
111373,126179,0,0.00
111374,126180,0,0.00


#### Total Count and Average Wire Amount Sent to External Users by Customer

In [48]:
# Create a groupby customerid to get the amount sent to external users for each customer
ext_recw = HR_wires1[HR_wires1['receiver_ext'] == 1]
sent_ext = pd.DataFrame(ext_recw.groupby(['sender_global_id']).agg(Num=('id receiver', np.count_nonzero), 
                                                              Avg=('wire value', np.mean))).reset_index().rename(columns={'sender_global_id': 'Global_id'})
sent_ext = find_custs(sent_ext)
sent_ext

,Global_id,Num,Avg
0,4,2,1305.0
1,18,1,7251.5
2,21,1,24687.5
3,23,2,15483.0
4,36,1,3841.5
...,...,...,...
116048,126177,0,0.0
116049,126178,0,0.0
116050,126179,0,0.0
116051,126180,0,0.0


#### Total Count and  Average Wire Amount Sent to Internal Users by Customer

In [49]:
# Create a groupby customerid to get the amount received from internal users for each customer
int_wires = HR_wires1[(HR_wires1['sender_ext'] == 0) & (HR_wires1['receiver_ext'] == 0)]
rec_int = pd.DataFrame(int_wires.groupby(['rec_global_id']).agg(Num=('id sender', np.count_nonzero), 
                                                              Avg=('wire value', np.mean))).reset_index().rename(columns={'rec_global_id': 'Global_id'})
rec_int = find_custs(rec_int)
rec_int

,Global_id,Num,Avg
0,1,1,1526.0
1,4,1,1240.0
2,6,1,7226.0
3,7,1,3780.0
4,18,1,6450.0
...,...,...,...
115512,126178,0,0.0
115513,126179,0,0.0
115514,126180,0,0.0
115515,126181,0,0.0


#### Total Count and Average Wire Amount Sent to Internal users (Customers, non-external)

In [50]:
# Create a groupby customerid to get the amount sent to internal users for each customer
sent_int = pd.DataFrame(int_wires.groupby(['sender_global_id']).agg(Num=('id receiver', np.count_nonzero), 
                                                              Avg=('wire value', np.mean))).reset_index().rename(columns={'sender_global_id': 'Global_id'})
sent_int = find_custs(sent_int)
sent_int

,Global_id,Num,Avg
0,4,2,3360.25
1,7,1,5046.00
2,15,1,1420.50
3,23,1,4040.50
4,46,1,4055.00
...,...,...,...
115511,126177,0,0.00
115512,126178,0,0.00
115513,126179,0,0.00
115514,126180,0,0.00


#### Total Count and Average Wire Amount Sent out of CA (Canada)

In [51]:
# Create a groupby to get the amount of money sent out of CA
out_CA = HR_wires1[(HR_wires1['out_CA'] == 1) & (HR_wires1['sender_ext'] == 0)]
out_CA_sent = pd.DataFrame(out_CA.groupby(['sender_global_id']).agg(Num=('id receiver', np.count_nonzero), 
                                                                  Avg=('wire value', np.mean))).reset_index().rename(columns={'sender_global_id': 'Global_id'})
out_CA_sent = find_custs(out_CA_sent)
out_CA_sent

,Global_id,Num,Avg
0,4,1,690.0
1,21,1,24687.5
2,23,1,28414.0
3,49,1,41340.0
4,70,1,8405.0
...,...,...,...
122433,126178,0,0.0
122434,126179,0,0.0
122435,126180,0,0.0
122436,126181,0,0.0


#### Total Count and Average Wire Amount Received into CA

In [52]:
# Create a groupby to get the amount of money received into CA
in_CA = HR_wires1[(HR_wires1['in_CA'] == 1) & (HR_wires1['receiver_ext'] == 0)]
in_CA_rec = pd.DataFrame(in_CA.groupby(['rec_global_id']).agg(Num=('id sender', np.count_nonzero), 
                                                                  Avg=('wire value', np.mean))).reset_index().rename(columns={'rec_global_id': 'Global_id'})
in_CA_rec = find_custs(in_CA_rec)
in_CA_rec

,Global_id,Num,Avg
0,77,1,6036.0
1,91,1,13790.0
2,111,1,2878.0
3,112,1,16852.0
4,133,1,1698.0
...,...,...,...
120514,126177,0,0.0
120515,126178,0,0.0
120516,126179,0,0.0
120517,126180,0,0.0


#### Total Count and Average Wire Amount Received Domestically

In [53]:
# create a groupby to get the amount received domestically
dom_wires = HR_wires1[(HR_wires1['out_CA'] == 0) & (HR_wires1['in_CA'] == 0) & (HR_wires1['receiver_ext'] == 0)]
dom_rece = pd.DataFrame(dom_wires.groupby(['rec_global_id']).agg(Num=('id sender', np.count_nonzero), 
                                                                  Avg=('wire value', np.mean))).reset_index().rename(columns={'rec_global_id': 'Global_id'})
dom_rece = find_custs(dom_rece)
dom_rece

,Global_id,Num,Avg
0,1,1,1526.000000
1,4,3,3623.000000
2,6,3,6547.000000
3,7,2,5787.500000
4,18,3,5642.833333
...,...,...,...
108953,126178,0,0.000000
108954,126179,0,0.000000
108955,126180,0,0.000000
108956,126181,0,0.000000


#### Total Count and Average Wire Amount Sent Domestically

In [54]:
# create a groupby to get the amount sent domestically
dom_wires2 = HR_wires1[(HR_wires1['out_CA'] == 0) & (HR_wires1['in_CA'] == 0) & (HR_wires1['sender_ext'] == 0)]
dom_sent = pd.DataFrame(dom_wires2.groupby(['sender_global_id']).agg(Num=('id receiver', np.count_nonzero), 
                                                                 Avg=('wire value', np.mean))).reset_index().rename(columns={'sender_global_id': 'Global_id'})
dom_sent = find_custs(dom_sent)
dom_sent

,Global_id,Num,Avg
0,4,3,2880.166667
1,7,1,5046.000000
2,15,1,1420.500000
3,18,1,7251.500000
4,23,2,3296.250000
...,...,...,...
111301,126177,0,0.000000
111302,126178,0,0.000000
111303,126179,0,0.000000
111304,126180,0,0.000000


In [55]:
# Merge the dataframes into a single dataframe
dfs = [sent_ext, rec_int, sent_int, out_CA_sent, in_CA_rec, dom_rece, dom_sent]

detail_info = received_ext.copy()

for i in range(len(dfs)):
    detail_info = pd.merge(detail_info, dfs[i], how='left', on='Global_id', suffixes=(i, i+1))

# Create a list of column names
cols = ['Global_id', 'Num_wire_rec_ext', 'Avg_wire_rec_ext', 
        'Num_wire_sent_ext', 'Avg_wire_sent_ext', 
        'Num_wire_rec_int', 'Avg_wire_rec_int', 
        'Num_wire_sent_int', 'Avg_wire_sent_int', 
        'Num_wire_out_CA', 'Avg_wire_out_CA', 
        'Num_wire_in_CA', 'Avg_wire_in_CA', 
        'Num_wire_rec_dom', 'Avg_wire_rec_dom', 
        'Num_wire_sent_dom', 'Avg_wire_sent_dom']

# Set the columns names to cols
detail_info.columns = cols

In [56]:
detail_info

,Global_id,Num_wire_rec_ext,Avg_wire_rec_ext,Num_wire_sent_ext,Avg_wire_sent_ext,Num_wire_rec_int,Avg_wire_rec_int,Num_wire_sent_int,Avg_wire_sent_int,Num_wire_out_CA,Avg_wire_out_CA,Num_wire_in_CA,Avg_wire_in_CA,Num_wire_rec_dom,Avg_wire_rec_dom,Num_wire_sent_dom,Avg_wire_sent_dom
0,4,2,4814.50,2,1305.0,1,1240.0,2,3360.25,1,690.0,0,0.0,3,3623.000000,3,2880.166667
1,6,2,6207.50,0,0.0,1,7226.0,0,0.00,0,0.0,0,0.0,3,6547.000000,0,0.000000
2,7,1,7795.00,0,0.0,1,3780.0,1,5046.00,0,0.0,0,0.0,2,5787.500000,1,5046.000000
3,18,2,5239.25,1,7251.5,1,6450.0,0,0.00,0,0.0,0,0.0,3,5642.833333,1,7251.500000
4,47,1,1974.00,0,0.0,1,1540.0,0,0.00,0,0.0,0,0.0,2,1757.000000,0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126178,126177,0,0.00,0,0.0,0,0.0,0,0.00,0,0.0,0,0.0,0,0.000000,0,0.000000
126179,126178,0,0.00,0,0.0,0,0.0,0,0.00,0,0.0,0,0.0,0,0.000000,0,0.000000
126180,126179,0,0.00,0,0.0,0,0.0,0,0.00,0,0.0,0,0.0,0,0.000000,0,0.000000
126181,126180,0,0.00,0,0.0,0,0.0,0,0.00,0,0.0,0,0.0,0,0.000000,0,0.000000


### Construct Dataframe for General Customer Emt Transactions

#### Total Count and Average Amount Emt Sent

In [57]:
# create a groupby to get the total amount of EMTs sent by each customer
sent = HR_emts1[HR_emts1['sender_ext'] == 0]
sent_w = pd.DataFrame(sent.groupby(['sender_global_id']).agg(Num=('id receiver', np.count_nonzero), 
                                                             Avg=('emt value', np.mean))).reset_index().rename(columns={'sender_global_id': 'Global_id'})
sent_w = find_custs(sent_w)
sent_w

,Global_id,Num,Avg
0,0,1,360.000000
1,2,1,159.000000
2,4,7,718.214286
3,5,9,220.611111
4,6,7,1374.928571
...,...,...,...
43714,126168,0,0.000000
43715,126169,0,0.000000
43716,126174,0,0.000000
43717,126178,0,0.000000


#### Total Count and Average EMT Amount Received

In [58]:
# create a groupby to get the total amount of EMTs recived by each customer
rec = HR_emts1[HR_emts1['receiver_ext'] == 0]
rec_w = pd.DataFrame(rec.groupby(['rec_global_id']).agg(Num=('id sender', np.count_nonzero), 
                                                             Avg=('emt value', np.mean))).reset_index().rename(columns={'rec_global_id': 'Global_id'})
rec_w = find_custs(rec_w)
rec_w

,Global_id,Num,Avg
0,3,1,69.000000
1,4,6,746.166667
2,5,15,220.066667
3,6,12,846.708333
4,7,2,633.000000
...,...,...,...
32047,126163,0,0.000000
32048,126167,0,0.000000
32049,126170,0,0.000000
32050,126175,0,0.000000


In [59]:
# Merge the dataframes together
gen_emtinfo = pd.merge(sent_w, rec_w, how='left', on='Global_id')
cols = ['Global_id', 'Num_emts_sent', 'Avg_emt_amt_sent', 'Num_emt_received', 'Avg_emt_amt_received']
gen_emtinfo.columns = cols
gen_emtinfo

,Global_id,Num_emts_sent,Avg_emt_amt_sent,Num_emt_received,Avg_emt_amt_received
0,0,1,360.000000,0,0.000000
1,2,1,159.000000,0,0.000000
2,4,7,718.214286,6,746.166667
3,5,9,220.611111,15,220.066667
4,6,7,1374.928571,12,846.708333
...,...,...,...,...,...
126178,126168,0,0.000000,1,252.000000
126179,126169,0,0.000000,1,202.000000
126180,126174,0,0.000000,1,5.000000
126181,126178,0,0.000000,2,54.500000


### Construct Dataframe for Specific Customer EMT Transactions

#### Total Count and Average EMT Amount Received from Externals

In [60]:
# Create a groupby customerid to get the amount received from external users for each customer
ext_sendw = HR_emts1[HR_emts1['sender_ext'] == 1]
received_ext = pd.DataFrame(ext_sendw.groupby(['rec_global_id']).agg(Num=('id sender', np.count_nonzero), 
                                                              Avg=('emt value', np.mean))).reset_index().rename(columns={'rec_global_id': 'Global_id'})

received_ext = find_custs(received_ext)
received_ext

,Global_id,Num,Avg
0,3,1,69.000000
1,4,4,138.625000
2,5,4,250.000000
3,6,7,878.714286
4,7,2,633.000000
...,...,...,...
60127,126170,0,0.000000
60128,126174,0,0.000000
60129,126175,0,0.000000
60130,126179,0,0.000000


#### Total Count and Average EMT Amount Sent to Externals

In [61]:
# Create a groupby customerid to get the amount sent to external users for each customer
ext_recw = HR_emts1[HR_emts1['receiver_ext'] == 1]
sent_ext = pd.DataFrame(ext_recw.groupby(['sender_global_id']).agg(Num=('id receiver', np.count_nonzero), 
                                                              Avg=('emt value', np.mean))).reset_index().rename(columns={'sender_global_id': 'Global_id'})
sent_ext = find_custs(sent_ext)
sent_ext

,Global_id,Num,Avg
0,2,1,159.000
1,4,2,577.000
2,5,4,227.375
3,6,4,2229.375
4,9,1,28.000
...,...,...,...
78829,126169,0,0.000
78830,126174,0,0.000
78831,126178,0,0.000
78832,126179,0,0.000


#### Total Count and Average EMT Amount Received from Internal Users

In [62]:
# Create a groupby customerid to get the amount received from internal users for each customer
int_wires = HR_emts1[(HR_emts1['sender_ext'] == 0) & (HR_emts1['receiver_ext'] == 0)]
rec_int = pd.DataFrame(int_wires.groupby(['rec_global_id']).agg(Num=('id sender', np.count_nonzero), 
                                                              Avg=('emt value', np.mean))).reset_index().rename(columns={'rec_global_id': 'Global_id'})
rec_int = find_custs(rec_int)
rec_int

,Global_id,Num,Avg
0,4,2,1961.250000
1,5,11,209.181818
2,6,5,801.900000
3,13,1,2444.000000
4,15,3,192.166667
...,...,...,...
65052,126170,0,0.000000
65053,126173,0,0.000000
65054,126175,0,0.000000
65055,126176,0,0.000000


#### Total Count and Average EMT Amount Sent to Internal Users

In [63]:
# Create a groupby customerid to get the amount sent to internal users for each customer
sent_int = pd.DataFrame(int_wires.groupby(['sender_global_id']).agg(Num=('id receiver', np.count_nonzero), 
                                                              Avg=('emt value', np.mean))).reset_index().rename(columns={'sender_global_id': 'Global_id'})
sent_int = find_custs(sent_int)
sent_int

,Global_id,Num,Avg
0,0,1,360.000000
1,4,5,774.700000
2,5,5,215.200000
3,6,3,235.666667
4,7,1,124.000000
...,...,...,...
65257,126175,0,0.000000
65258,126176,0,0.000000
65259,126177,0,0.000000
65260,126178,0,0.000000


In [64]:
# Merge the dataframes into a single dataframe
dfs = [sent_ext, rec_int, sent_int]

detail_emtinfo = received_ext.copy()

for i in range(len(dfs)):
    detail_emtinfo = pd.merge(detail_emtinfo, dfs[i], how='left', on='Global_id', suffixes=(i, i+1))

# Create a list of column names
cols = ['Global_id', 'Num_emt_rec_ext', 'Avg_emt_rec_ext', 
        'Num_emt_sent_ext', 'Avg_emt_sent_ext', 
        'Num_emt_rec_int', 'Avg_emt_rec_int', 
        'Num_emt_sent_int', 'Avg_emt_sent_int']

# Set the columns names to cols
detail_emtinfo.columns = cols

In [65]:
detail_emtinfo

,Global_id,Num_emt_rec_ext,Avg_emt_rec_ext,Num_emt_sent_ext,Avg_emt_sent_ext,Num_emt_rec_int,Avg_emt_rec_int,Num_emt_sent_int,Avg_emt_sent_int
0,3,1,69.000000,0,0.000,0,0.000000,0,0.000000
1,4,4,138.625000,2,577.000,2,1961.250000,5,774.700000
2,5,4,250.000000,4,227.375,11,209.181818,5,215.200000
3,6,7,878.714286,4,2229.375,5,801.900000,3,235.666667
4,7,2,633.000000,0,0.000,0,0.000000,1,124.000000
...,...,...,...,...,...,...,...,...,...
126178,126170,0,0.000000,1,965.000,0,0.000000,0,0.000000
126179,126174,0,0.000000,0,0.000,1,5.000000,0,0.000000
126180,126175,0,0.000000,1,450.000,0,0.000000,0,0.000000
126181,126179,0,0.000000,0,0.000,1,121.000000,0,0.000000


### Construct Dataframe for Cash Transactions

In [66]:
# Create a groupby to get statistics for the cash transactions
cash_info = pd.DataFrame(HR_cash1.groupby(['Global_id', 'type']).agg(Num=('trxn_id', np.count_nonzero), 
                                                                   Avg=('amount', np.mean))).reset_index()
# Pivot the DataFrame
cash_info = cash_info.pivot(index='Global_id', columns='type', values=['Num', 'Avg']).reset_index()
cash_info.columns = ['_'.join(col) for col in cash_info.columns]

# fill np.nan with 0
cash_info = cash_info.fillna(0).rename(columns={'Global_id_':'Global_id'})
cash_info

,Global_id,Num_deposit,Num_withdrawal,Avg_deposit,Avg_withdrawal
0,5,0.0,2.0,0.000000,1265.000000
1,15,3.0,2.0,4090.000000,4667.500000
2,18,14.0,3.0,8818.571429,5018.333333
3,23,10.0,4.0,7808.500000,5725.000000
4,36,0.0,1.0,0.000000,2060.000000
...,...,...,...,...,...
15592,126147,0.0,4.0,0.000000,1252.500000
15593,126150,1.0,0.0,1140.000000,0.000000
15594,126151,2.0,0.0,1210.000000,0.000000
15595,126159,1.0,0.0,1625.000000,0.000000


Get customers who had no cash transactions and append them to the table.

In [67]:
# get the customers that were recorded
recorded = set(cash_info['Global_id'])

# find the set difference between the non-exteral customer gids and the recorded ones
not_recorded = list(no_xcusts.difference(recorded))

# create 0s to fill in the num and sum columns
fills = [0] * len(not_recorded)

# create a dataframe and append to existing dataframe
nr_df = pd.DataFrame({'Global_id': not_recorded, 
                      'Num_deposit': fills, 
                      'Num_withdrawal': fills, 
                      'Avg_deposit': fills, 
                      'Avg_withdrawal': fills})

cash_info = pd.concat([cash_info, nr_df])
cash_info

,Global_id,Num_deposit,Num_withdrawal,Avg_deposit,Avg_withdrawal
0,5,0.0,2.0,0.000000,1265.000000
1,15,3.0,2.0,4090.000000,4667.500000
2,18,14.0,3.0,8818.571429,5018.333333
3,23,10.0,4.0,7808.500000,5725.000000
4,36,0.0,1.0,0.000000,2060.000000
...,...,...,...,...,...
110581,126177,0.0,0.0,0.000000,0.000000
110582,126178,0.0,0.0,0.000000,0.000000
110583,126179,0.0,0.0,0.000000,0.000000
110584,126181,0.0,0.0,0.000000,0.000000


## Extracting External Individuals data

### Get Wire related transactions

In [68]:
# Get a list of only external customers
ext_custs = set(custs_df[custs_df['ext'] == 1]['Global_id'])
len(ext_custs)

69875

In [69]:
# create a function to find the external customers that did not have any amount for the given transaction
def find_xcusts(df):
    """
    """
    # get the customers that were recorded
    recorded = set(df['Global_id'])
    
    # find the set difference between the total ext_custs and the recorded ones
    not_recorded = list(ext_custs.difference(recorded))
    
    # create 0s to fill in the num and sum columns
    fills = [0] * len(not_recorded)
    
    # create a dataframe and append to existing dataframe
    nr_df = pd.DataFrame({'Global_id': not_recorded, 'Num': fills, 'Avg': fills})
    
    return pd.concat([df, nr_df])

#### Total Count and Average Wire Amount Sent by External Users

In [70]:
# create a groupby to get the total amount of wires sent by each external customer
sent = HR_wires1[HR_wires1['sender_ext'] == 1]
sent_w = pd.DataFrame(sent.groupby(['sender_global_id']).agg(Num=('id receiver', np.count_nonzero), 
                                                             Avg=('wire value', np.mean))).reset_index().rename(columns={'sender_global_id': 'Global_id'})
sent_w = find_xcusts(sent_w)
sent_w

,Global_id,Num,Avg
0,126184,1,1546.0
1,126199,1,1046.0
2,126200,1,1444.0
3,126201,1,3710.0
4,126208,1,38450.0
...,...,...,...
55905,196051,0,0.0
55906,196052,0,0.0
55907,196054,0,0.0
55908,196056,0,0.0


#### Total Count and Average Wire Amount Received by External User

In [71]:
# create a groupby to get the total amount of wires recived by each external customer
rec = HR_wires1[HR_wires1['receiver_ext'] == 1]
rec_w = pd.DataFrame(rec.groupby(['rec_global_id']).agg(Num=('id sender', np.count_nonzero), 
                                                             Avg=('wire value', np.mean))).reset_index().rename(columns={'rec_global_id': 'Global_id'})
rec_w = find_xcusts(rec_w)
rec_w

,Global_id,Num,Avg
0,126201,1,4011.000000
1,126224,3,10718.666667
2,126233,1,13440.000000
3,126239,1,12868.000000
4,126240,1,849.500000
...,...,...,...
59931,196053,0,0.000000
59932,196054,0,0.000000
59933,196055,0,0.000000
59934,196056,0,0.000000


In [72]:
# Merge the dataframes together
wire_extinfo = pd.merge(sent_w, rec_w, how='left', on='Global_id')
cols = ['Global_id', 'Num_wires_sent', 'Avg_wire_amt_sent', 'Num_wires_received', 'Avg_wire_amt_received']
wire_extinfo.columns = cols
wire_extinfo

,Global_id,Num_wires_sent,Avg_wire_amt_sent,Num_wires_received,Avg_wire_amt_received
0,126184,1,1546.0,0,0.0
1,126199,1,1046.0,0,0.0
2,126200,1,1444.0,0,0.0
3,126201,1,3710.0,1,4011.0
4,126208,1,38450.0,0,0.0
...,...,...,...,...,...
69870,196051,0,0.0,0,0.0
69871,196052,0,0.0,0,0.0
69872,196054,0,0.0,0,0.0
69873,196056,0,0.0,0,0.0


### Get Emt related transactions

#### Total Count and Average EMT AMount sent by External Users

In [73]:
# create a groupby to get the total amount of emt sent by each external
sent = HR_emts1[HR_emts1['sender_ext'] == 1]
sent_e = pd.DataFrame(sent.groupby(['sender_global_id']).agg(Num=('id receiver', np.count_nonzero), 
                                                             Avg=('emt value', np.mean))).reset_index().rename(columns={'sender_global_id': 'Global_id'})
sent_e = find_xcusts(sent_e)
sent_e

,Global_id,Num,Avg
0,126183,3,21.666667
1,126185,1,1379.000000
2,126186,1,57.000000
3,126187,2,344.000000
4,126188,3,9.166667
...,...,...,...
21283,131056,0,0.000000
21284,131065,0,0.000000
21285,131068,0,0.000000
21286,131070,0,0.000000


#### Total Count and Average EMT Amount Received by Customer

In [74]:
# create a groupby to get the total amount of emts received by each customer
rec = HR_emts1[HR_emts1['receiver_ext'] == 1]
rec_e = pd.DataFrame(rec.groupby(['rec_global_id']).agg(Num=('id sender', np.count_nonzero), 
                                                             Avg=('emt value', np.mean))).reset_index().rename(columns={'rec_global_id': 'Global_id'})
rec_e = find_xcusts(rec_e)
rec_e

,Global_id,Num,Avg
0,126183,1,32.0
1,126187,2,295.0
2,126188,2,5.0
3,126191,1,595.0
4,126195,1,44.0
...,...,...,...
33045,131061,0,0.0
33046,131062,0,0.0
33047,131063,0,0.0
33048,131065,0,0.0


In [75]:
# Merge the dataframes together
emt_extinfo = pd.merge(sent_e, rec_e, how='left', on='Global_id')
cols = ['Global_id', 'Num_emts_sent', 'Avg_emt_amt_sent', 'Num_emts_received', 'Avg_emt_amt_received']
emt_extinfo.columns = cols
emt_extinfo

,Global_id,Num_emts_sent,Avg_emt_amt_sent,Num_emts_received,Avg_emt_amt_received
0,126183,3,21.666667,1,32.00
1,126185,1,1379.000000,0,0.00
2,126186,1,57.000000,0,0.00
3,126187,2,344.000000,2,295.00
4,126188,3,9.166667,2,5.00
...,...,...,...,...,...
69870,131056,0,0.000000,2,220.25
69871,131065,0,0.000000,0,0.00
69872,131068,0,0.000000,1,272.00
69873,131070,0,0.000000,1,1396.00


### Get Country of External Users

In [76]:
# Get external user country data
ext_cty = cust_ctrys[cust_ctrys['ext'] == 1].drop(columns='ext')
ext_cty

,Global_id,Country
1,153141,CA
5,184775,CA
6,152221,CA
7,191772,CN
9,141282,CA
...,...,...
196053,196051,Missing
196054,196052,Missing
196055,196054,Missing
196056,196056,Missing


# Encoding Customer Table Features

#### Get dummies for Gender Variable

In [77]:
# One-hot encode the Gender variable
HR_kyc1 = pd.get_dummies(HR_kyc, columns=['Gender'], dtype=int)
HR_kyc1

,Name,Occupation,Age,Tenure,cust_id,label,Global_id,Gender_female,Gender_male,Gender_other
0,JENNIFER WELLS,Architect,45.0,13.0,CUST82758793,0,102123,1,0,0
1,ANTHONY ADAMS,Musician,52.0,8.0,CUST69248708,0,83128,0,1,0
2,DENISE LEWIS,Jewelry Dealer,43.0,11.0,CUST67222818,0,80244,1,0,0
3,KYLE EDWARDS,Real Estate Broker,39.0,21.0,CUST76401392,1,93124,0,1,0
4,BECKY SUTTON,Sommelier,26.0,8.0,CUST41866546,0,44548,1,0,0
...,...,...,...,...,...,...,...,...,...,...
126178,WANG MIN,Private Equity Fund Manager,37.0,12.0,CUST67655265,1,80822,1,0,0
126179,THOMAS YOUNG,Software Developer,46.0,3.0,CUST23014082,0,18375,0,1,0
126180,CASEY JONES,Miner,31.0,10.0,CUST17691251,0,10812,0,1,0
126181,NICOLE-CÉCILE LEBLANC,Unknown,21.0,3.0,CUST26444112,0,23226,1,0,0


#### Encode Occupations variable

In [78]:
# get list of unique occupations
occupations = list(HR_kyc1['Occupation'].unique())
occupations.append(np.nan)

# use sk-learn label encoder to encode the occupation variable using the occupations found in the kyc table
l_encoder = LabelEncoder()

# Create new column for encoded occupation
l_encoder.fit(occupations)
HR_kyc1['Occupation_num'] = l_encoder.transform(HR_kyc1['Occupation'])

# Drop unnecessary columns 
HR_kyc1 = HR_kyc1.drop(columns=['Name', 'cust_id', 'Gender_other']).set_index('Global_id').reset_index()
HR_kyc1

,Global_id,Occupation,Age,Tenure,label,Gender_female,Gender_male,Occupation_num
0,102123,Architect,45.0,13.0,0,1,0,8
1,83128,Musician,52.0,8.0,0,0,1,146
2,80244,Jewelry Dealer,43.0,11.0,0,1,0,115
3,93124,Real Estate Broker,39.0,21.0,1,0,1,195
4,44548,Sommelier,26.0,8.0,0,1,0,212
...,...,...,...,...,...,...,...,...
126178,80822,Private Equity Fund Manager,37.0,12.0,1,1,0,186
126179,18375,Software Developer,46.0,3.0,0,0,1,211
126180,10812,Miner,31.0,10.0,0,0,1,143
126181,23226,Unknown,21.0,3.0,0,1,0,235


# Merge Dataframes for Customer info

#### `General Customer Transaction Info` DF

There are only two unique values for country after merging the dataframes together, 'CA' and 'Missing'. Since this dataframe includes all customers that are non-external, we can assume that individuals with 'Missing' have no wire transactions, and that the country is still CA. Therefore we will drop the country column as there is no addition information gained from the variable.

In [79]:
# Create general customer info dataframe
gci_dfs = [gen_emtinfo, cash_info]
gen_custinfo = gen_wireinfo.copy()

for i in range(len(gci_dfs)):
    gen_custinfo = pd.merge(gen_custinfo, gci_dfs[i], how='left', on='Global_id')

gen_custinfo 

,Global_id,Num_wires_sent,Avg_wire_amt_sent,Num_wires_received,Avg_wire_amt_received,Num_emts_sent,Avg_emt_amt_sent,Num_emt_received,Avg_emt_amt_received,Num_deposit,Num_withdrawal,Avg_deposit,Avg_withdrawal
0,4,4,2332.625,3,3623.000000,7,718.214286,6,746.166667,0.0,0.0,0.000000,0.000000
1,7,1,5046.000,2,5787.500000,1,124.000000,2,633.000000,0.0,0.0,0.000000,0.000000
2,15,1,1420.500,0,0.000000,9,895.888889,6,255.000000,3.0,2.0,4090.000000,4667.500000
3,18,1,7251.500,3,5642.833333,4,1185.750000,8,422.125000,14.0,3.0,8818.571429,5018.333333
4,21,1,24687.500,0,0.000000,0,0.000000,2,1019.750000,0.0,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
126178,126177,0,0.000,0,0.000000,1,170.000000,2,289.500000,0.0,0.0,0.000000,0.000000
126179,126178,0,0.000,0,0.000000,0,0.000000,2,54.500000,0.0,0.0,0.000000,0.000000
126180,126179,0,0.000,0,0.000000,0,0.000000,1,121.000000,0.0,0.0,0.000000,0.000000
126181,126180,0,0.000,0,0.000000,6,218.000000,8,235.062500,9.0,1.0,8737.222222,8245.000000


####  `Detailed Customer Transaction Info` DF

In [80]:
addons = [detail_info, detail_emtinfo]
cust_details = gen_custinfo.copy()

for i in range(len(addons)):
    cust_details = pd.merge(cust_details, addons[i], how='left', on='Global_id')
    
cust_details

,Global_id,Num_wires_sent,Avg_wire_amt_sent,Num_wires_received,Avg_wire_amt_received,Num_emts_sent,Avg_emt_amt_sent,Num_emt_received,Avg_emt_amt_received,Num_deposit,...,Num_wire_sent_dom,Avg_wire_sent_dom,Num_emt_rec_ext,Avg_emt_rec_ext,Num_emt_sent_ext,Avg_emt_sent_ext,Num_emt_rec_int,Avg_emt_rec_int,Num_emt_sent_int,Avg_emt_sent_int
0,4,4,2332.625,3,3623.000000,7,718.214286,6,746.166667,0.0,...,3,2880.166667,4,138.625000,2,577.0,2,1961.250000,5,774.700000
1,7,1,5046.000,2,5787.500000,1,124.000000,2,633.000000,0.0,...,1,5046.000000,2,633.000000,0,0.0,0,0.000000,1,124.000000
2,15,1,1420.500,0,0.000000,9,895.888889,6,255.000000,3.0,...,1,1420.500000,3,317.833333,3,586.0,3,192.166667,6,1050.833333
3,18,1,7251.500,3,5642.833333,4,1185.750000,8,422.125000,14.0,...,1,7251.500000,2,51.000000,2,2098.0,6,545.833333,2,273.500000
4,21,1,24687.500,0,0.000000,0,0.000000,2,1019.750000,0.0,...,0,0.000000,2,1019.750000,0,0.0,0,0.000000,0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126178,126177,0,0.000,0,0.000000,1,170.000000,2,289.500000,0.0,...,0,0.000000,1,344.000000,1,170.0,1,235.000000,0,0.000000
126179,126178,0,0.000,0,0.000000,0,0.000000,2,54.500000,0.0,...,0,0.000000,1,67.000000,0,0.0,1,42.000000,0,0.000000
126180,126179,0,0.000,0,0.000000,0,0.000000,1,121.000000,0.0,...,0,0.000000,0,0.000000,0,0.0,1,121.000000,0,0.000000
126181,126180,0,0.000,0,0.000000,6,218.000000,8,235.062500,9.0,...,0,0.000000,6,199.333333,1,302.0,2,342.250000,5,201.200000


#### `General Customer Info` DF

In [81]:
# Merge gen_custinfo with HR_kyc1 to get all general customer info in one dataframe
cust_info = pd.merge(HR_kyc1, gen_custinfo, on='Global_id')
cust_info

,Global_id,Occupation,Age,Tenure,label,Gender_female,Gender_male,Occupation_num,Num_wires_sent,Avg_wire_amt_sent,Num_wires_received,Avg_wire_amt_received,Num_emts_sent,Avg_emt_amt_sent,Num_emt_received,Avg_emt_amt_received,Num_deposit,Num_withdrawal,Avg_deposit,Avg_withdrawal
0,102123,Architect,45.0,13.0,0,1,0,8,2,1985.75,0,0.000000,7,2058.857143,8,1435.562500,0.0,0.0,0.000000,0.00
1,83128,Musician,52.0,8.0,0,0,1,146,0,0.00,0,0.000000,0,0.000000,1,84.000000,0.0,0.0,0.000000,0.00
2,80244,Jewelry Dealer,43.0,11.0,0,1,0,115,2,14131.50,4,5657.375000,5,898.800000,13,502.961538,2.0,0.0,1917.500000,0.00
3,93124,Real Estate Broker,39.0,21.0,1,0,1,195,2,5214.50,3,7965.166667,5,229.500000,8,2286.125000,9.0,2.0,7568.333333,9792.50
4,44548,Sommelier,26.0,8.0,0,1,0,212,0,0.00,0,0.000000,0,0.000000,1,10.000000,0.0,0.0,0.000000,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126178,80822,Private Equity Fund Manager,37.0,12.0,1,1,0,186,0,0.00,0,0.000000,4,31.250000,3,31.333333,10.0,4.0,8767.000000,8248.75
126179,18375,Software Developer,46.0,3.0,0,0,1,211,0,0.00,0,0.000000,3,263.166667,13,252.346154,0.0,0.0,0.000000,0.00
126180,10812,Miner,31.0,10.0,0,0,1,143,0,0.00,0,0.000000,0,0.000000,1,634.000000,0.0,0.0,0.000000,0.00
126181,23226,Unknown,21.0,3.0,0,1,0,235,0,0.00,1,1466.000000,8,590.000000,7,478.214286,0.0,0.0,0.000000,0.00


#### `Detailed Customer Info` DF

In [82]:
# Merge gen_custinfo with HR_kyc1 to get all general customer info in one dataframe
detailed_cust_info = pd.merge(HR_kyc1, cust_details, on='Global_id')

# Create a copy with Occupation and one without
dci = detailed_cust_info.copy()

detailed_cust_info = detailed_cust_info.drop(columns=['Occupation'])
detailed_cust_info

,Global_id,Age,Tenure,label,Gender_female,Gender_male,Occupation_num,Num_wires_sent,Avg_wire_amt_sent,Num_wires_received,...,Num_wire_sent_dom,Avg_wire_sent_dom,Num_emt_rec_ext,Avg_emt_rec_ext,Num_emt_sent_ext,Avg_emt_sent_ext,Num_emt_rec_int,Avg_emt_rec_int,Num_emt_sent_int,Avg_emt_sent_int
0,102123,45.0,13.0,0,1,0,8,2,1985.75,0,...,2,1985.75,2,448.000000,3,2023.666667,6,1764.750000,4,2085.250
1,83128,52.0,8.0,0,0,1,146,0,0.00,0,...,0,0.00,1,84.000000,0,0.000000,0,0.000000,0,0.000
2,80244,43.0,11.0,0,1,0,115,2,14131.50,4,...,1,1305.00,6,279.666667,1,168.000000,7,694.357143,4,1081.500
3,93124,39.0,21.0,1,0,1,195,2,5214.50,3,...,2,5214.50,7,2195.285714,1,55.000000,1,2922.000000,4,273.125
4,44548,26.0,8.0,0,1,0,212,0,0.00,0,...,0,0.00,1,10.000000,0,0.000000,0,0.000000,0,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126178,80822,37.0,12.0,1,1,0,186,0,0.00,0,...,0,0.00,2,31.000000,0,0.000000,1,32.000000,4,31.250
126179,18375,46.0,3.0,0,0,1,211,0,0.00,0,...,0,0.00,6,308.250000,1,220.000000,7,204.428571,2,284.750
126180,10812,31.0,10.0,0,0,1,143,0,0.00,0,...,0,0.00,1,634.000000,0,0.000000,0,0.000000,0,0.000
126181,23226,21.0,3.0,0,1,0,235,0,0.00,1,...,0,0.00,3,407.000000,0,0.000000,4,531.625000,8,590.000


#### `External Users` DF

In [83]:
# Merge the wire and emt tranasctions dataframes into one dataframe for external users
ext_df = pd.merge(wire_extinfo, emt_extinfo, on='Global_id')
ext_info = pd.merge(ext_df, ext_cty, on='Global_id')

# Get dummies for Country column
external_info = pd.get_dummies(ext_info, columns=['Country'], dtype=int)
external_info

,Global_id,Num_wires_sent,Avg_wire_amt_sent,Num_wires_received,Avg_wire_amt_received,Num_emts_sent,Avg_emt_amt_sent,Num_emts_received,Avg_emt_amt_received,Country_AU,Country_CA,Country_CN,Country_GE,Country_IN,Country_MX,Country_Missing,Country_RU,Country_SA,Country_UK,Country_US
0,126184,1,1546.0,0,0.0,0,0.000000,0,0.00,1,0,0,0,0,0,0,0,0,0,0
1,126199,1,1046.0,0,0.0,9,419.888889,10,797.35,0,1,0,0,0,0,0,0,0,0,0
2,126200,1,1444.0,0,0.0,0,0.000000,0,0.00,0,0,1,0,0,0,0,0,0,0,0
3,126201,1,3710.0,1,4011.0,8,1643.000000,5,840.40,0,1,0,0,0,0,0,0,0,0,0
4,126208,1,38450.0,0,0.0,0,0.000000,0,0.00,0,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69870,196051,0,0.0,0,0.0,1,403.000000,2,494.50,0,0,0,0,0,0,1,0,0,0,0
69871,196052,0,0.0,0,0.0,1,1128.000000,2,1065.25,0,0,0,0,0,0,1,0,0,0,0
69872,196054,0,0.0,0,0.0,2,377.500000,2,166.50,0,0,0,0,0,0,1,0,0,0,0
69873,196056,0,0.0,0,0.0,3,9.333333,0,0.00,0,0,0,0,0,0,1,0,0,0,0


## Clean Transaction DataFrames

### Clean Wire and Emt Transactions DataFrame

In [84]:
# Create a function to clean the transaction dataframes
def clean_trxns(df):
    """
    """
    # Make a copy of the wire datafram
    c = df.copy()

    # Fill np.nan in Gender with 'Missing' and get dummies
    c[['sender_Gender', 'receiver_Gender']] = c[['sender_Gender', 'receiver_Gender']].fillna('Missing')
    c1 = pd.get_dummies(c, columns=['sender_Gender', 'receiver_Gender'], dtype=int)

    # Encode Occupation with previous encoder
    c1['sender_Occupation_num'] = l_encoder.transform(c1['sender_Occupation'])
    c1['rec_Occupation_num'] = l_encoder.transform(c1['receiver_Occupation'])

    # Fill missing values for Age, Tenure and label with -1
    c_nulls = ['sender_Age', 'receiver_Age', 'sender_Tenure', 'receiver_Tenure', 'sender_label', 'receiver_label']
    c1[c_nulls] = c1[c_nulls].fillna(-1)

    # Drop unnecessary variables
    cleaned = c1.drop(columns=['id sender', 'id receiver', 
                               'name sender', 'name receiver', 
                               'sender_Name', 'receiver_Name', 
                               'sender_Occupation', 'receiver_Occupation'])
    
    return cleaned

In [85]:
# Clean the wire transactions dataframe
wires = clean_trxns(HR_wires1)
wires

,wire value,trxn_id,sender_Age,sender_Tenure,sender_label,receiver_Age,receiver_Tenure,receiver_label,sender_global_id,rec_global_id,...,sender_Gender_Missing,sender_Gender_female,sender_Gender_male,sender_Gender_other,receiver_Gender_Missing,receiver_Gender_female,receiver_Gender_male,receiver_Gender_other,sender_Occupation_num,rec_Occupation_num
0,1267.0,LWCS42954834,34.0,0.0,0.0,45.0,21.0,0.0,46393,118403,...,0,1,0,0,0,0,1,0,8,53
1,8591.0,NTTG55749308,38.0,5.0,1.0,-1.0,-1.0,-1.0,101584,183844,...,0,1,0,0,1,0,0,0,91,250
2,1480.5,IXVD84599097,-1.0,-1.0,-1.0,56.0,21.0,0.0,184551,104548,...,1,0,0,0,0,1,0,0,250,51
3,1587.0,SLBV29462341,22.0,3.0,0.0,-1.0,-1.0,-1.0,71803,162608,...,0,0,1,0,1,0,0,0,217,250
4,1546.0,ERLU26785367,36.0,0.0,1.0,61.0,13.0,0.0,117672,9197,...,0,0,1,0,0,1,0,0,19,164
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48287,6059.5,LRRP66624765,-1.0,-1.0,-1.0,35.0,12.0,0.0,142262,107141,...,1,0,0,0,0,0,1,0,250,247
48288,5067.0,KVQK50168638,25.0,0.0,0.0,-1.0,-1.0,-1.0,112004,155051,...,0,1,0,0,1,0,0,0,217,250
48289,18874.0,IUIP17370739,44.0,1.0,0.0,-1.0,-1.0,-1.0,69903,167767,...,0,1,0,0,1,0,0,0,216,250
48290,4084.0,ZHVK78574815,34.0,5.0,1.0,42.0,0.0,0.0,29953,96357,...,0,1,0,0,0,0,0,1,228,193


In [86]:
# Clean the emt transactions dataframe
emts = clean_trxns(HR_emts1)

# create binary variable for whether there is an emt message for the transaction
emts.loc[(emts['emt message'].isnull()), 'has_msg'] = 0
emts['has_msg'] = emts['has_msg'].fillna(1).astype(int)
emts = emts.drop(columns='emt message')
emts

,emt value,trxn_id,sender_Age,sender_Tenure,sender_label,receiver_Age,receiver_Tenure,receiver_label,sender_global_id,rec_global_id,...,sender_Gender_female,sender_Gender_male,sender_Gender_other,receiver_Gender_Missing,receiver_Gender_female,receiver_Gender_male,receiver_Gender_other,sender_Occupation_num,rec_Occupation_num,has_msg
0,1170.5,RAUG63886259,-1.0,-1.0,-1.0,37.0,6.0,0.0,166925,69450,...,0,0,0,0,0,1,0,250,111,0
1,46.0,WPXP45854083,34.0,8.0,0.0,-1.0,-1.0,-1.0,68825,155054,...,0,1,0,1,0,0,0,110,250,0
2,480.0,TRNT55099512,69.0,14.0,0.0,22.0,4.0,0.0,24521,6745,...,0,0,1,0,0,1,0,188,52,0
3,735.0,YSNV62579819,44.0,14.0,1.0,49.0,10.0,0.0,77547,23015,...,1,0,0,0,1,0,0,8,62,0
4,540.0,MZYI28216959,-1.0,-1.0,-1.0,48.0,3.0,0.0,156121,78536,...,0,0,0,0,1,0,0,250,120,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
318895,682.0,XAOG83079223,34.0,3.0,0.0,42.0,16.0,0.0,62944,91980,...,1,0,0,0,0,1,0,64,60,0
318896,119.0,USHN74907347,-1.0,-1.0,-1.0,27.0,1.0,0.0,156186,91130,...,0,0,0,0,1,0,0,250,37,0
318897,208.0,VXES44436032,46.0,9.0,0.0,35.0,0.0,0.0,3063,82331,...,1,0,0,0,0,1,0,73,235,0
318898,150.0,LTUK21435620,-1.0,-1.0,-1.0,29.0,5.0,0.0,155038,112810,...,0,0,0,0,0,1,0,250,195,1


## Clean cash transactions

In [87]:
# Create a copy of HR_cash1
c = HR_cash1.copy()

# Get dummies for transaction type and gender of customer
c = pd.get_dummies(c, columns=['type', 'Gender'], dtype=int)

# Encode Occupation using l_encoder
c['Occupation_num'] = l_encoder.transform(c['Occupation'])

# Drop unnecessary columns
cash = c.drop(columns=['cust_id', 'Name', 'Occupation', 'type_withdrawal', 'Gender_other'])
cash

,amount,trxn_id,Age,Tenure,label,sports,gamble,sales,valuables,estate,...,student,code,news,stats,vague,Global_id,type_deposit,Gender_female,Gender_male,Occupation_num
0,4800,BFMG48785876,33.0,0.0,0,0,0,0,0,1,...,0,0,0,0,0,96095,1,1,0,194
1,7420,FUKV94845036,48.0,17.0,1,0,0,0,0,0,...,0,0,0,0,0,102552,0,0,1,185
2,5595,NUZO58830551,39.0,16.0,1,0,0,0,0,0,...,0,0,0,0,0,45375,1,1,0,185
3,1600,ZOSP34629709,43.0,7.0,0,0,0,0,0,0,...,0,0,0,0,0,39239,0,1,0,8
4,1055,HQUJ43887606,30.0,4.0,0,0,0,0,0,0,...,0,0,0,0,0,80360,0,1,0,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90224,1140,JOQU43611104,48.0,2.0,0,0,0,0,1,0,...,0,0,0,0,0,85640,1,1,0,4
90225,6870,LTBH81014009,34.0,1.0,1,0,0,0,0,0,...,0,0,0,0,0,105558,1,0,1,187
90226,8740,GGHM25093698,22.0,4.0,0,0,0,0,0,0,...,0,0,0,0,0,42954,0,0,1,51
90227,5750,CNXP31340871,31.0,3.0,1,0,0,0,0,0,...,0,0,0,0,0,23527,0,1,0,91


# Final Dataframes

Transaction Dataframes
- `wires`: wire trxn info
- `emts`: emt trxn info
- `cash`: cash trxn info

Customer/External Dataframes
- `cust_info`: general customer info
- `detailed_cust_info`: detailed customer info
- `dci`: detailed customer info with Original Occupation Variable
- `external_info`: external user info

# Export the Final Dataframes

In [88]:
# export the dataframes
wires.to_csv('wires.csv')
emts.to_csv('emts.csv')
cash.to_csv('cash.csv')
cust_info.to_csv('cust_info.csv')
detailed_cust_info.to_csv('detailed_cust_info.csv')
external_info.to_csv('external_info.csv')
dci.to_csv('dci.csv')